# Modelo definitivo

CHONA ME DIJO QUE USARA LOS DATOS SIN PCA Y QUE HAGA UN POOLING GRANDE AL PRINCIPIO DE LA CNN PARA ACHICAR LOS DATOS DE UNA CUANDO ENTRAN. 
ADEMÁS, TENGO QUE HACER CONV2D EN VES DE 1D Y TENGO QUE AGREGARLE LOS NUEVOS DATOS. ADEMÁS TENGO QUE USAR EL PRETRAINED MODEL RESNET QUE USO 
EL INDIO EN EL VIDEO PARA TENER YO YA HECHO EL COSO Y CON LOS WEIGHTS Y QUE NO ME TRADE TANTO EL ENTRENAMIENTO. YO LE CAMBIARÍA LO DEL POOL AL 
PRINCIPIO Y LO ADAPTARÍA AL FINAL DE LA RED PARA QUE SE AJUSTE A MIS DATOS.
- TENGO QUE CORRER EL CODIGO PARA EL DATASET DE NUEVO PERO SIN EL PCA 
- VOLVER A ARREGLAR LA CNN PARA QUE ADMITA LA VIEJA INPUT SHAPE
- USAR UN MODELO RESNET PREENTRENADO CON DATOS COHERENTES (PREFERENTEMENTE EL QUE USÓ EL INDIO)
- INCORPORAR LO QUE ME DIJO CHONA DE CONV2D Y UN POOLING PARA QUE SE ADAPTE A RESNET (AdaptiveAvgPool2d - https://stackoverflow.com/questions/61955991/change-input-shape-dimensions-for-resnet-model-pytorch)

CODIGOS DEL INDIO:
https://github.com/talhaanwarch/youtube-tutorials/blob/main/eeg-conv2d.ipynb
https://github.com/talhaanwarch/youtube-tutorials/blob/main/eeg_epilepsy.ipynb

In [4]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D, GlobalAveragePooling2D, MaxPooling2D, Dropout, Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
#import tensorflowjs
import pandas as pd
import numpy as np
import os
import gc

In [2]:
'''import wandb
import random
wandb.login(key="3750a6378bb91e331366851db2db2f8b674797f6")'''

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\47575909/.netrc


True

In [22]:
'''# Inicializa una ejecución de WandB
wandb.init(project="SinapsIA", entity="schajrisgaratiagustina")

# Configura hiperparámetros
config = wandb.config
config.learning_rate = 0.001
config.batch_size = 64'''

Problem at: C:\Users\47575909\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\sdk\wandb_init.py 848 getcaller


KeyboardInterrupt: 

In [2]:
#X = np.load('C:\\Users\\47575909\\Desktop\\X_12.npy')
#y = np.load('C:\\Users\\47575909\\Desktop\\y_12.npy')


In [3]:
#X.shape, y.shape

((1790, 224, 224, 3), (1790,))

In [4]:
# X = X.reshape(1790, 244, 244, 3)

In [5]:
# X = X[:,:224,:224,:]

In [6]:
#X.shape

(1790, 224, 224, 3)

In [10]:
input_tensor = Input(shape=(224, 224, 3))

#Cargo el modelo ResNet50 preentrenado
base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor)

#Congelo las capas del modelo base para que no se actualicen en el entrenamiento
for layer in base_model.layers:
    layer.trainable = False

#Agrego GlobalAveragePooling2D para adaptar la salida del modelo base
#x = GlobalAveragePooling2D()(base_model.output)

#Agrego capas adicionales para la clasificación de mis datos
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(500, activation='relu')(x)  # Puedes usar Dense en lugar de Conv2D aquí
x = Dense(200, activation='relu')(x)  # Capas Dense en lugar de MaxPooling2D
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

'''#x = base_model.output
#x = GlobalAveragePooling2D()(x)
x = Conv2D(filters=16, kernel_size=3, activation='relu')(base_model.output)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=16, kernel_size=3, activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Flatten()(x)
x = Dense(8, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4, activation='relu')(x)

output = Dense(1, activation='softmax')(x)'''

#Junto todo y creo el modelo
model = Model(inputs=input_tensor, outputs=output)

In [11]:
#COMPILO
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
'''def data_generator():
    for i in range(10):
        X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_{i+1}.npy')
        y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_{i+1}.npy')
        yield X, y

# https://stackoverflow.com/a/39271995'''

In [12]:
for i in range(2):
    X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_{i+1}.npy')
    y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_{i+1}.npy')

    #DIVIDO EN TRAIN, TEST Y EVALUATION
    #Divido los datos de train de los de test y evaluation (70/30)
    X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

    #Divido los datos de test de los de evaluation (50/50 del 30% anterior)
    X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)

    model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

    #EVALÚO
    eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
    del X
    del y
    del X_val
    del X_test
    del y_val
    del y_test
    del X_train
    del X_testVal
    del y_train
    del y_testVal

    gc.collect()

    print("Accuracy : ", eval_acc)
    print("Loss : ", eval_loss)

Epoch 1/10
67/67 [==============================] - 94s 1s/step - loss: 0.7068 - accuracy: 0.5939 - val_loss: 0.5599 - val_accuracy: 0.7112
Epoch 2/10
67/67 [==============================] - 92s 1s/step - loss: 0.5636 - accuracy: 0.7108 - val_loss: 0.4319 - val_accuracy: 0.8053
Epoch 3/10
67/67 [==============================] - 93s 1s/step - loss: 0.4386 - accuracy: 0.7953 - val_loss: 0.3639 - val_accuracy: 0.8425
Epoch 4/10
67/67 [==============================] - 92s 1s/step - loss: 0.3929 - accuracy: 0.8254 - val_loss: 0.3357 - val_accuracy: 0.8512
Epoch 5/10
67/67 [==============================] - 92s 1s/step - loss: 0.3436 - accuracy: 0.8535 - val_loss: 0.5232 - val_accuracy: 0.7681
Epoch 6/10
67/67 [==============================] - 92s 1s/step - loss: 0.3375 - accuracy: 0.8559 - val_loss: 0.3681 - val_accuracy: 0.8490
Epoch 7/10
67/67 [==============================] - 92s 1s/step - loss: 0.2921 - accuracy: 0.8808 - val_loss: 0.3095 - val_accuracy: 0.8600
Epoch 8/10
67/67 [==

In [4]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_1.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_1.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

#Divido los datos de test de los de evaluation (50/50 del 30% anterior)
X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del X_test
del y_val
del y_test
del X_train
del X_testVal
del y_train
del y_testVal

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

Epoch 1/10
67/67 [==============================] - 92s 1s/step - loss: 0.7293 - accuracy: 0.5981 - val_loss: 0.5933 - val_accuracy: 0.5930
Epoch 2/10
67/67 [==============================] - 90s 1s/step - loss: 0.5897 - accuracy: 0.5981 - val_loss: 0.5433 - val_accuracy: 0.5930
Epoch 3/10
67/67 [==============================] - 90s 1s/step - loss: 0.5377 - accuracy: 0.5981 - val_loss: 0.5412 - val_accuracy: 0.5930
Epoch 4/10
67/67 [==============================] - 90s 1s/step - loss: 0.4352 - accuracy: 0.5981 - val_loss: 0.3728 - val_accuracy: 0.5930
Epoch 5/10
67/67 [==============================] - 91s 1s/step - loss: 0.3762 - accuracy: 0.5981 - val_loss: 0.3271 - val_accuracy: 0.5930
Epoch 6/10
67/67 [==============================] - 91s 1s/step - loss: 0.3453 - accuracy: 0.5981 - val_loss: 0.2976 - val_accuracy: 0.5930
Epoch 7/10
67/67 [==============================] - 91s 1s/step - loss: 0.2641 - accuracy: 0.5981 - val_loss: 0.2667 - val_accuracy: 0.5930
Epoch 8/10
67/67 [==

In [ ]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_2.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_2.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

#Divido los datos de test de los de evaluation (50/50 del 30% anterior)
X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del X_test
del y_val
del y_test
del X_train
del X_testVal
del y_train
del y_testVal

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

Epoch 1/10
69/69 [==============================] - 95s 1s/step - loss: 0.7541 - accuracy: 0.5739 - val_loss: 0.6813 - val_accuracy: 0.5610
Epoch 2/10
69/69 [==============================] - 94s 1s/step - loss: 0.6744 - accuracy: 0.5739 - val_loss: 0.6593 - val_accuracy: 0.5610
Epoch 3/10
69/69 [==============================] - 94s 1s/step - loss: 0.6401 - accuracy: 0.5739 - val_loss: 0.6028 - val_accuracy: 0.5610
Epoch 4/10
69/69 [==============================] - 94s 1s/step - loss: 0.5012 - accuracy: 0.5739 - val_loss: 0.3825 - val_accuracy: 0.5610
Epoch 5/10
69/69 [==============================] - 94s 1s/step - loss: 0.4186 - accuracy: 0.5739 - val_loss: 0.3541 - val_accuracy: 0.5610
Epoch 6/10
69/69 [==============================] - 94s 1s/step - loss: 0.3299 - accuracy: 0.5739 - val_loss: 0.3243 - val_accuracy: 0.5610
Epoch 7/10
69/69 [==============================] - 94s 1s/step - loss: 0.2814 - accuracy: 0.5739 - val_loss: 0.2928 - val_accuracy: 0.5610
Epoch 8/10
69/69 [==

In [ ]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_3.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_3.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

#Divido los datos de test de los de evaluation (50/50 del 30% anterior)
X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del X_test
del y_val
del y_test
del X_train
del X_testVal
del y_train
del y_testVal

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

In [ ]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\4.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_4.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

#Divido los datos de test de los de evaluation (50/50 del 30% anterior)
X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del X_test
del y_val
del y_test
del X_train
del X_testVal
del y_train
del y_testVal

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

In [20]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_5.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_5.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

#Divido los datos de test de los de evaluation (50/50 del 30% anterior)
X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del X_test
del y_val
del y_test
del X_train
del X_testVal
del y_train
del y_testVal

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

Epoch 1/10
67/67 [==============================] - 92s 1s/step - loss: 0.5757 - accuracy: 0.7854 - val_loss: 0.4351 - val_accuracy: 0.8256
Epoch 2/10
67/67 [==============================] - 92s 1s/step - loss: 0.4536 - accuracy: 0.8233 - val_loss: 0.4261 - val_accuracy: 0.8389
Epoch 3/10
67/67 [==============================] - 95s 1s/step - loss: 0.4192 - accuracy: 0.8360 - val_loss: 0.3793 - val_accuracy: 0.8543
Epoch 4/10
67/67 [==============================] - 95s 1s/step - loss: 0.3709 - accuracy: 0.8554 - val_loss: 0.3328 - val_accuracy: 0.8675
Epoch 5/10
67/67 [==============================] - 94s 1s/step - loss: 0.3368 - accuracy: 0.8644 - val_loss: 0.3439 - val_accuracy: 0.8631
Epoch 6/10
67/67 [==============================] - 94s 1s/step - loss: 0.2939 - accuracy: 0.8705 - val_loss: 0.2581 - val_accuracy: 0.8698
Epoch 7/10
67/67 [==============================] - 93s 1s/step - loss: 0.2949 - accuracy: 0.8785 - val_loss: 0.2751 - val_accuracy: 0.9095
Epoch 8/10
67/67 [==

In [21]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_6.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_6.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

#Divido los datos de test de los de evaluation (50/50 del 30% anterior)
X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del X_test
del y_val
del y_test
del X_train
del X_testVal
del y_train
del y_testVal

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

Epoch 1/10
69/69 [==============================] - 96s 1s/step - loss: 0.6441 - accuracy: 0.6706 - val_loss: 0.5397 - val_accuracy: 0.7152
Epoch 2/10
69/69 [==============================] - 95s 1s/step - loss: 0.4934 - accuracy: 0.7550 - val_loss: 0.4325 - val_accuracy: 0.8201
Epoch 3/10
69/69 [==============================] - 97s 1s/step - loss: 0.3618 - accuracy: 0.8472 - val_loss: 0.3611 - val_accuracy: 0.8415
Epoch 4/10
69/69 [==============================] - 98s 1s/step - loss: 0.3056 - accuracy: 0.8633 - val_loss: 0.3109 - val_accuracy: 0.8758
Epoch 5/10
69/69 [==============================] - 96s 1s/step - loss: 0.2520 - accuracy: 0.8995 - val_loss: 0.2387 - val_accuracy: 0.9143
Epoch 6/10
69/69 [==============================] - 95s 1s/step - loss: 0.2182 - accuracy: 0.9128 - val_loss: 0.2726 - val_accuracy: 0.8737
Epoch 7/10
69/69 [==============================] - 95s 1s/step - loss: 0.1804 - accuracy: 0.9289 - val_loss: 0.2494 - val_accuracy: 0.9122
Epoch 8/10
69/69 [==

In [22]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_7.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_7.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

#Divido los datos de test de los de evaluation (50/50 del 30% anterior)
X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del X_test
del y_val
del y_test
del X_train
del X_testVal
del y_train
del y_testVal

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

Epoch 1/10
69/69 [==============================] - 95s 1s/step - loss: 0.7363 - accuracy: 0.5797 - val_loss: 0.6536 - val_accuracy: 0.6368
Epoch 2/10
69/69 [==============================] - 95s 1s/step - loss: 0.6134 - accuracy: 0.6506 - val_loss: 0.5977 - val_accuracy: 0.6923
Epoch 3/10
69/69 [==============================] - 96s 1s/step - loss: 0.5747 - accuracy: 0.6909 - val_loss: 0.5541 - val_accuracy: 0.7158
Epoch 4/10
69/69 [==============================] - 96s 1s/step - loss: 0.5352 - accuracy: 0.7262 - val_loss: 0.5425 - val_accuracy: 0.7350
Epoch 5/10
69/69 [==============================] - 96s 1s/step - loss: 0.4950 - accuracy: 0.7550 - val_loss: 0.5246 - val_accuracy: 0.7329
Epoch 6/10
69/69 [==============================] - 95s 1s/step - loss: 0.4618 - accuracy: 0.7830 - val_loss: 0.4995 - val_accuracy: 0.7585
Epoch 7/10
69/69 [==============================] - 95s 1s/step - loss: 0.4260 - accuracy: 0.7908 - val_loss: 0.4671 - val_accuracy: 0.7671
Epoch 8/10
69/69 [==

In [23]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_8.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_8.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

#Divido los datos de test de los de evaluation (50/50 del 30% anterior)
X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del X_test
del y_val
del y_test
del X_train
del X_testVal
del y_train
del y_testVal

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

Epoch 1/10
54/54 [==============================] - 73s 1s/step - loss: 0.4225 - accuracy: 0.8527 - val_loss: 0.3893 - val_accuracy: 0.8583
Epoch 2/10
54/54 [==============================] - 76s 1s/step - loss: 0.2759 - accuracy: 0.8802 - val_loss: 0.2955 - val_accuracy: 0.8583
Epoch 3/10
54/54 [==============================] - 73s 1s/step - loss: 0.2335 - accuracy: 0.9012 - val_loss: 0.2585 - val_accuracy: 0.8910
Epoch 4/10
54/54 [==============================] - 73s 1s/step - loss: 0.2085 - accuracy: 0.9153 - val_loss: 0.2280 - val_accuracy: 0.9019
Epoch 5/10
54/54 [==============================] - 73s 1s/step - loss: 0.1871 - accuracy: 0.9269 - val_loss: 0.2260 - val_accuracy: 0.8992
Epoch 6/10
54/54 [==============================] - 73s 1s/step - loss: 0.1396 - accuracy: 0.9445 - val_loss: 0.2083 - val_accuracy: 0.9210
Epoch 7/10
54/54 [==============================] - 73s 1s/step - loss: 0.1158 - accuracy: 0.9562 - val_loss: 0.1998 - val_accuracy: 0.9183
Epoch 8/10
54/54 [==

In [24]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_9.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_9.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

#Divido los datos de test de los de evaluation (50/50 del 30% anterior)
X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del X_test
del y_val
del y_test
del X_train
del X_testVal
del y_train
del y_testVal

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

Epoch 1/10
33/33 [==============================] - 45s 1s/step - loss: 0.0444 - accuracy: 0.9847 - val_loss: 2.1004e-04 - val_accuracy: 1.0000
Epoch 2/10
33/33 [==============================] - 44s 1s/step - loss: 3.6120e-04 - accuracy: 1.0000 - val_loss: 8.4590e-06 - val_accuracy: 1.0000
Epoch 3/10
33/33 [==============================] - 44s 1s/step - loss: 3.1966e-05 - accuracy: 1.0000 - val_loss: 3.0846e-06 - val_accuracy: 1.0000
Epoch 4/10
33/33 [==============================] - 44s 1s/step - loss: 1.1802e-05 - accuracy: 1.0000 - val_loss: 2.3010e-06 - val_accuracy: 1.0000
Epoch 5/10
33/33 [==============================] - 44s 1s/step - loss: 1.1715e-05 - accuracy: 1.0000 - val_loss: 2.2025e-06 - val_accuracy: 1.0000
Epoch 6/10
33/33 [==============================] - 44s 1s/step - loss: 1.1707e-05 - accuracy: 1.0000 - val_loss: 2.0364e-06 - val_accuracy: 1.0000
Epoch 7/10
33/33 [==============================] - 44s 1s/step - loss: 1.4096e-05 - accuracy: 1.0000 - val_loss: 1.

In [25]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_10.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_10.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

#Divido los datos de test de los de evaluation (50/50 del 30% anterior)
X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del X_test
del y_val
del y_test
del X_train
del X_testVal
del y_train
del y_testVal

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

Epoch 1/10
73/73 [==============================] - 101s 1s/step - loss: 1.8479e-05 - accuracy: 1.0000 - val_loss: 6.7133e-06 - val_accuracy: 1.0000
Epoch 2/10
73/73 [==============================] - 100s 1s/step - loss: 1.0340e-05 - accuracy: 1.0000 - val_loss: 5.0228e-06 - val_accuracy: 1.0000
Epoch 3/10
73/73 [==============================] - 100s 1s/step - loss: 7.1389e-06 - accuracy: 1.0000 - val_loss: 4.5908e-06 - val_accuracy: 1.0000
Epoch 4/10
73/73 [==============================] - 100s 1s/step - loss: 5.8212e-05 - accuracy: 1.0000 - val_loss: 1.1343e-06 - val_accuracy: 1.0000
Epoch 5/10
73/73 [==============================] - 100s 1s/step - loss: 5.0606e-06 - accuracy: 1.0000 - val_loss: 7.9667e-07 - val_accuracy: 1.0000
Epoch 6/10
73/73 [==============================] - 101s 1s/step - loss: 8.8287e-06 - accuracy: 1.0000 - val_loss: 7.4836e-07 - val_accuracy: 1.0000
Epoch 7/10
73/73 [==============================] - 100s 1s/step - loss: 1.2100e-05 - accuracy: 1.0000 - v

In [7]:
'''X = np.load('C:\\Users\\47575909\\Desktop\\Dataset\\X_1.npy')
y = np.load('C:\\Users\\47575909\\Desktop\\Dataset\\y_1.npy')'''

In [ ]:
'''#DIVIDO EN TRAIN, TEST Y EVALUATION
    #Divido los datos de train de los de test y evaluation (70/30)
    X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

    #Divido los datos de test de los de evaluation (50/50 del 30% anterior)
    X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)'''

In [8]:
'''model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=[wandb.keras.WandbCallback())

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)

wandb.log({"eval_loss": test_loss, "eval_acc": test_acc})'''

Epoch 1/10
 9/96 [=>............................] - ETA: 1:39 - loss: 0.9005 - accuracy: 0.6111

KeyboardInterrupt: 

In [ ]:
'''print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss'''

In [6]:
'''def model_train(archivo_x, archivo_y):
    X = np.load(archivo_x)
    y = np.load(archivo_y)

    #DIVIDO EN TRAIN, TEST Y EVALUATION
    #Divido los datos de train de los de test y evaluation (70/30)
    X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

    #Divido los datos de test de los de evaluation (50/50 del 30% anterior)
    X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)

    
    model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=[wandb.keras.WandbCallback())

    #EVALÚO
    eval_loss, eval_acc = model.evaluate(X_val, y_val)

    wandb.log({"eval_loss": test_loss, "eval_acc": test_acc})

    #VEO LA PERFORMANCE
    print("Accuracy : ", eval_acc)
    print("Loss : ", eval_loss)'''

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (1637144083.py, line 13)

In [25]:
'''directorio = 'C:\\Users\\47575909\\Desktop\\Dataset'
num_archivos = 10

for i in range(1, num_archivos + 1):
    archivo_x = os.path.join(directorio, f'X_{i}.npy')
    archivo_y = os.path.join(directorio, f'y_{i}.npy')

    model_train(archivo_x, archivo_y)'''

ValueError: Found input variables with inconsistent numbers of samples: [8181, 2727]

In [ ]:
'''X = np.load('C:\\Users\\47575909\\Desktop\\X.npy')
y = np.load('C:\\Users\\47575909\\Desktop\\y.npy')'''

In [ ]:
'''#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

#Divido los datos de test de los de evaluation (50/50 del 30% anterior)
X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)'''

In [ ]:
'''#ENTRENO
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
test_loss, test_acc = model.evaluate(X_val, y_val)'''

In [ ]:
'''#VEO LA PERFORMANCE
print("Accuracy : ", test_acc)
print("Loss : ", test_loss)'''

In [27]:
prueba_normal = np.load('C:\\Users\\47575909\\Desktop\\X_predict_epilepsy.npy')

#HAGO UNA PREDICCIÓN
prediction = model.predict(prueba_normal)

probability = np.mean(prediction)
#epilepsy_prob = prediction[:, 1]
#print("Probabilidad de tener epilepsia:", epilepsy_prob)

print(probability)

10/10 [==============================] - 10s 1s/step
6.024863e-08


##### Decirle a chona que el indio usa ResNet-26D y que keras no lo tiene ese. Que hago?...
- Uso otro modelo de ResNet
- Uso las liberías pytorch y timm que si tienen el ResNet-26D

# Exportamos el modelo
(Con tensorflow.js)

In [26]:
#guardamos el modelo
model.save('modelo_SinapsIA.h5')

C:\Users\47575909\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
'''#Exportamos el modelooo
tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model --signature_name=serving_default --saved_model_tags=serve /ruta/al/modelo/saved_model /ruta/de/destino/
#che, no entiendo lo de las rutas'''

In [ ]:
//HABRÍA QUE HACER ESTO PARA QUE ME DEVUELVA EN LA WEB SOLO LA PROBABILIDAD QUE YO QUIERO (la probabilidad de tener epilepsia)

// Obtener datos de entrada (pueden ser simulados en este ejemplo)
const datosEntrada = obtenerDatosDeEntrada();

// Cargar el modelo
const modelo = await cargarModelo();

// Realizar predicciones
const predicciones = modelo.predict(tf.tensor(datosEntrada));

// Mostrar la probabilidad de tener epilepsia
const probabilidadEpilepsia = predicciones.mean();
console.log('Probabilidad de tener epilepsia:', probabilidadEpilepsia);